이 실습은 Runpod에서 A100 SXM GPU 1개로 진행되었습니다.

## 1. 필요한 패키지 설치

In [24]:
%pip install "torch==2.4.0"
%pip install "transformers==4.45.1" "datasets==3.0.1" "accelerate==0.34.2" "trl==0.11.1" "peft==0.13.0"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch==2.2.0 torchvision==0.17.0+cu121 --extra-index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy==1.26.4

/bin/bash: line 1: 2: No such file or directory
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
print(np.__version__)

1.26.4


## 2. 데이터 전처리

In [1]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

In [9]:
from datasets import load_dataset

# 1. 허깅페이스 허브에서 데이터셋 로드
dataset = load_dataset("julietz/law_term_2", split="train")

# 2. 전체 데이터 크기만 출력
print("전체 데이터 크기:", len(dataset))

# 3. train/test 분할 비율 설정
test_ratio = 0.2

train_data = []
test_data = []

# 4. 전체 데이터의 인덱스를 train/test로 분할
data_indices = list(range(len(dataset)))
test_size = int(len(data_indices) * test_ratio)

test_data = data_indices[:test_size]
train_data = data_indices[test_size:]

# 5. OpenAI format으로 데이터 변환을 위한 함수
def format_data(sample):
    return {
        "messages": [
            {
                "role": "system",
                "content": sample["system_prompt"],
            },
            {
                "role": "user",
                "content": sample["user_prompt"],
            },
            {
                "role": "assistant",
                "content": str(sample["assistant"])
            },
        ],
    }

# 6. 분할된 데이터를 OpenAI format으로 변환
train_dataset = [format_data(dataset[i]) for i in train_data]
test_dataset = [format_data(dataset[i]) for i in test_data]

# 7. 최종 데이터셋 크기 출력
print(f"\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개")

전체 데이터 크기: 1000

전체 데이터 분할 결과: Train 800개, Test 200개


In [10]:
# 임의의 345번 데이터 출력
train_dataset[345]["messages"]

[{'role': 'system',
  'content': '당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 역할은 아래 두 가지 작업을 수행하는 것입니다:\n\n1. 주어진 "법령 용어(법령정의사전)"를 기반으로, 고객이 자연스럽게 질문할 법한 문장을 생성하세요.\n2. 주어진 "법령 용어"의 "사전식 풀이"를 바탕으로, 고객에게 친절하고 정확하게 설명하는 답변을 작성하세요.\n\n---\n\n[1] 질문 문장 생성 규칙:\n- 말투는 자연스럽고 일상적인 질문 형태로 작성할 것\n- 문장은 짧고 간결하게, 대략적인 수준으로 작성할 것\n- 다음과 같은 표현을 활용할 것: "뭐예요?", "뭔가요?", "설명해 주세요."\n- 용어만 나열하지 말고, 반드시 문장 형태로 만들 것\n- 사람을 지칭하는 경우에는 "누구인가요?", "누구를 말하는 건가요?"처럼 질문할 것\n- 반드시 질문 문장만 출력하고, 다른 설명이나 안내 문구는 출력하지 말 것\n\n예시:\n- 입력: 1인당 GNI → 출력: 1인당 GNI가 뭔가요?\n- 입력: 1종 금융투자업자 → 출력: 1종 금융투자업자는 누구를 말하는 건가요?\n\n---\n\n[2] 정의 설명 및 출처 처리 규칙:\n- 두 번째 작업의 결과는 반드시 파이썬의 dictionary 형식으로 출력해야 합니다.\n- 큰따옴표(") 안에 작은따옴표(\')나 다른 따옴표를 넣지 마세요. JSON 파싱 오류가 발생합니다.\n- 해당 항목이 없을 경우에는 "없음"이라는 표현 대신, 빈 문자열("") 또는 빈 리스트([])를 사용해야 합니다.\n\n요구 조건:\n- 딱딱하거나 기계적인 문장은 피하고, 자연스럽고 격식 있는 설명을 작성하세요.\n- 어려운 정의는 2~3문장으로 풀어 설명해 주세요. 단, 중복된 표현은 삼가세요.\n- 정의 설명 결과는 "법령용어정의"라는 키에 담아야 합니다.\n\n출처 관련 규칙:\n- 출처가 없을 경우:\n  {"법령용어정의": "...", "출처유무": false}\n\

In [11]:
# 리스트 형태에서 다시 Dataset 객체로 변경
print(type(train_dataset))
print(type(test_dataset))
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)
print(type(train_dataset))
print(type(test_dataset))

<class 'list'>
<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [12]:
train_dataset[0]

{'messages': [{'content': '당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.\n\n당신의 역할은 아래 두 가지 작업을 수행하는 것입니다:\n\n1. 주어진 "법령 용어(법령정의사전)"를 기반으로, 고객이 자연스럽게 질문할 법한 문장을 생성하세요.\n2. 주어진 "법령 용어"의 "사전식 풀이"를 바탕으로, 고객에게 친절하고 정확하게 설명하는 답변을 작성하세요.\n\n---\n\n[1] 질문 문장 생성 규칙:\n- 말투는 자연스럽고 일상적인 질문 형태로 작성할 것\n- 문장은 짧고 간결하게, 대략적인 수준으로 작성할 것\n- 다음과 같은 표현을 활용할 것: "뭐예요?", "뭔가요?", "설명해 주세요."\n- 용어만 나열하지 말고, 반드시 문장 형태로 만들 것\n- 사람을 지칭하는 경우에는 "누구인가요?", "누구를 말하는 건가요?"처럼 질문할 것\n- 반드시 질문 문장만 출력하고, 다른 설명이나 안내 문구는 출력하지 말 것\n\n예시:\n- 입력: 1인당 GNI → 출력: 1인당 GNI가 뭔가요?\n- 입력: 1종 금융투자업자 → 출력: 1종 금융투자업자는 누구를 말하는 건가요?\n\n---\n\n[2] 정의 설명 및 출처 처리 규칙:\n- 두 번째 작업의 결과는 반드시 파이썬의 dictionary 형식으로 출력해야 합니다.\n- 큰따옴표(") 안에 작은따옴표(\')나 다른 따옴표를 넣지 마세요. JSON 파싱 오류가 발생합니다.\n- 해당 항목이 없을 경우에는 "없음"이라는 표현 대신, 빈 문자열("") 또는 빈 리스트([])를 사용해야 합니다.\n\n요구 조건:\n- 딱딱하거나 기계적인 문장은 피하고, 자연스럽고 격식 있는 설명을 작성하세요.\n- 어려운 정의는 2~3문장으로 풀어 설명해 주세요. 단, 중복된 표현은 삼가세요.\n- 정의 설명 결과는 "법령용어정의"라는 키에 담아야 합니다.\n\n출처 관련 규칙:\n- 출처가 없을 경우:\n  {"법령용어정의": "...", "출처유무": false}\n\n- 출처가 

In [13]:
test_dataset.save_to_disk("test_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/200 [00:00<?, ? examples/s]

In [7]:
from huggingface_hub import login
login("key")

In [15]:
from datasets import Dataset, DatasetDict
dataset_dict = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

## 2. 모델 로드 및 템플릿 적용

In [8]:
# 허깅페이스 모델 ID
model_id = "NCSOFT/Llama-VARCO-8B-Instruct" 

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
# 템플릿 적용
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.

당신의 역할은 아래 두 가지 작업을 수행하는 것입니다:

1. 주어진 "법령 용어(법령정의사전)"를 기반으로, 고객이 자연스럽게 질문할 법한 문장을 생성하세요.
2. 주어진 "법령 용어"의 "사전식 풀이"를 바탕으로, 고객에게 친절하고 정확하게 설명하는 답변을 작성하세요.

---

[1] 질문 문장 생성 규칙:
- 말투는 자연스럽고 일상적인 질문 형태로 작성할 것
- 문장은 짧고 간결하게, 대략적인 수준으로 작성할 것
- 다음과 같은 표현을 활용할 것: "뭐예요?", "뭔가요?", "설명해 주세요."
- 용어만 나열하지 말고, 반드시 문장 형태로 만들 것
- 사람을 지칭하는 경우에는 "누구인가요?", "누구를 말하는 건가요?"처럼 질문할 것
- 반드시 질문 문장만 출력하고, 다른 설명이나 안내 문구는 출력하지 말 것

예시:
- 입력: 1인당 GNI → 출력: 1인당 GNI가 뭔가요?
- 입력: 1종 금융투자업자 → 출력: 1종 금융투자업자는 누구를 말하는 건가요?

---

[2] 정의 설명 및 출처 처리 규칙:
- 두 번째 작업의 결과는 반드시 파이썬의 dictionary 형식으로 출력해야 합니다.
- 큰따옴표(") 안에 작은따옴표(')나 다른 따옴표를 넣지 마세요. JSON 파싱 오류가 발생합니다.
- 해당 항목이 없을 경우에는 "없음"이라는 표현 대신, 빈 문자열("") 또는 빈 리스트([])를 사용해야 합니다.

요구 조건:
- 딱딱하거나 기계적인 문장은 피하고, 자연스럽고 격식 있는 설명을 작성하세요.
- 어려운 정의는 2~3문장으로 풀어 설명해 주세요. 단, 중복된 표현은 삼가세요.
- 정의 설명 결과는 "법령용어정의"라는 키에 담아야 합니다.

출처 관련 규칙:
- 출처가 없을 경우:
  {"법령용어정의": "...", "출처유무": false}

- 출처가 있을 경

## 3. LoRA와 SFTConfig 설정

In [10]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

In [11]:
args = SFTConfig(
    output_dir="llama3-8b-qa-ko",           # 저장될 디렉토리와 저장소 ID
    num_train_epochs=3,                      # 학습할 총 에포크 수 
    per_device_train_batch_size=2,           # GPU당 배치 크기
    gradient_accumulation_steps=2,           # 그래디언트 누적 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",               # 최적화기
    logging_steps=10,                        # 로그 기록 주기
    save_strategy="steps",                   # 저장 전략
    save_steps=50,                           # 저장 주기
    bf16=True,                              # bfloat16 사용
    learning_rate=1e-4,                     # 학습률
    max_grad_norm=0.3,                      # 그래디언트 클리핑
    warmup_ratio=0.03,                      # 워밍업 비율
    lr_scheduler_type="constant",           # 고정 학습률
    push_to_hub=False,                      # 허브 업로드 안 함
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=None
)

## 4. 학습 중 전처리 함수: collate_fn

In [12]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    for example in batch:
        messages = example["messages"]

        # LLaMA 3 채팅 템플릿 적용 (시작 토큰 포함)
        prompt = "<|begin_of_text|>"
        for msg in messages:
            role = msg["role"]
            content = msg["content"].strip()
            prompt += f"<|start_header_id|>{role}<|end_header_id|>\n{content}<|eot_id|>"

        # 마지막 assistant 메시지는 응답으로 간주하고 레이블에 포함
        text = prompt.strip()

        # 토큰화
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        labels = [-100] * len(input_ids)

        # assistant 응답의 시작 위치 찾기
        assistant_header = "<|start_header_id|>assistant<|end_header_id|>\n"
        assistant_tokens = tokenizer.encode(assistant_header, add_special_tokens=False)
        eot_token = "<|eot_id|>"
        eot_tokens = tokenizer.encode(eot_token, add_special_tokens=False)

        # 레이블 범위 지정
        i = 0
        while i <= len(input_ids) - len(assistant_tokens):
            if input_ids[i:i + len(assistant_tokens)] == assistant_tokens:
                start = i + len(assistant_tokens)
                end = start
                while end <= len(input_ids) - len(eot_tokens):
                    if input_ids[end:end + len(eot_tokens)] == eot_tokens:
                        break
                    end += 1
                for j in range(start, end):
                    labels[j] = input_ids[j]
                for j in range(end, end + len(eot_tokens)):
                    labels[j] = input_ids[j]  # <|eot_id|> 토큰도 포함
                break
            i += 1

        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)

    # 패딩 처리
    max_length = max(len(ids) for ids in new_batch["input_ids"])
    for i in range(len(new_batch["input_ids"])):
        pad_len = max_length - len(new_batch["input_ids"][i])
        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * pad_len)
        new_batch["attention_mask"][i].extend([0] * pad_len)
        new_batch["labels"][i].extend([-100] * pad_len)

    for k in new_batch:
        new_batch[k] = torch.tensor(new_batch[k])

    return new_batch

```python
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

시스템 프롬프트<|eot_id|><|start_header_id|>user<|end_header_id|>

유저 프롬프트<|eot_id|><|start_header_id|>assistant<|end_header|>LLM의 답변<|eot_id|>
```

In [13]:
# 데이터의 최대 길이 제한
max_seq_length=8192

# collate_fn 테스트 (배치 크기 1. 즉, 데이터 1개에 대해서 전처리를 진행해본다.)
example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 1107])
어텐션 마스크 형태: torch.Size([1, 1107])
레이블 형태: torch.Size([1, 1107])


In [14]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())

입력에 대한 정수 인코딩 결과:
[128000, 128006, 9125, 128007, 198, 65895, 83628, 34804, 107849, 104366, 120974, 18359, 107472, 102783, 245, 103850, 229, 55348, 18918, 53017, 44005, 55348, 116425, 20565, 80052, 382, 65895, 83628, 21028, 103135, 48936, 34804, 116100, 103405, 109521, 121002, 18359, 125201, 44005, 123263, 1473, 16, 13, 56773, 125441, 330, 101661, 104366, 107032, 32179, 7, 101661, 104366, 30381, 21028, 56154, 66965, 10143, 18918, 126470, 43139, 11, 116534, 13094, 124110, 25941, 107496, 58901, 109760, 48936, 107849, 24486, 54535, 115096, 53017, 92245, 627, 17, 13, 56773, 125441, 330, 101661, 104366, 107032, 32179, 1, 21028, 330, 56154, 66965, 77437, 107745, 103618, 1, 18918, 82818, 120378, 43139, 11, 116534, 102244, 108280, 104834, 101360, 127923, 102893, 114942, 44005, 111964, 18359, 114839, 92245, 382, 45464, 58, 16, 60, 109760, 54535, 41953, 53017, 111850, 117035, 512, 12, 101264, 105292, 16969, 124110, 25941, 107496, 35495, 84656, 57002, 103684, 109760, 106612, 87472, 17835, 114839, 

In [15]:
# 디코딩된 input_ids 출력
decoded_text = tokenizer.decode(
    batch["input_ids"][0].tolist(),
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\ninput_ids 디코딩 결과:")
print(decoded_text)


input_ids 디코딩 결과:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.

당신의 역할은 아래 두 가지 작업을 수행하는 것입니다:

1. 주어진 "법령 용어(법령정의사전)"를 기반으로, 고객이 자연스럽게 질문할 법한 문장을 생성하세요.
2. 주어진 "법령 용어"의 "사전식 풀이"를 바탕으로, 고객에게 친절하고 정확하게 설명하는 답변을 작성하세요.

---

[1] 질문 문장 생성 규칙:
- 말투는 자연스럽고 일상적인 질문 형태로 작성할 것
- 문장은 짧고 간결하게, 대략적인 수준으로 작성할 것
- 다음과 같은 표현을 활용할 것: "뭐예요?", "뭔가요?", "설명해 주세요."
- 용어만 나열하지 말고, 반드시 문장 형태로 만들 것
- 사람을 지칭하는 경우에는 "누구인가요?", "누구를 말하는 건가요?"처럼 질문할 것
- 반드시 질문 문장만 출력하고, 다른 설명이나 안내 문구는 출력하지 말 것

예시:
- 입력: 1인당 GNI → 출력: 1인당 GNI가 뭔가요?
- 입력: 1종 금융투자업자 → 출력: 1종 금융투자업자는 누구를 말하는 건가요?

---

[2] 정의 설명 및 출처 처리 규칙:
- 두 번째 작업의 결과는 반드시 파이썬의 dictionary 형식으로 출력해야 합니다.
- 큰따옴표(") 안에 작은따옴표(')나 다른 따옴표를 넣지 마세요. JSON 파싱 오류가 발생합니다.
- 해당 항목이 없을 경우에는 "없음"이라는 표현 대신, 빈 문자열("") 또는 빈 리스트([])를 사용해야 합니다.

요구 조건:
- 딱딱하거나 기계적인 문장은 피하고, 자연스럽고 격식 있는 설명을 작성하세요.
- 어려운 정의는 2~3문장으로 풀어 설명해 주세요. 단, 중복된 표현은 삼가세요.
- 정의 설명 결과는 "법령용어정의"라는 키에 담아야 합니다.

출처 관련 규칙:
- 출처가 없을 경우:
  {"법령용어정의": "...", "출처유무": 

In [16]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

레이블에 대한 정수 인코딩 결과:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

In [17]:
# -100이 아닌 부분만 골라 디코딩
label_ids = [token_id for token_id in batch["labels"][0].tolist() if token_id != -100]

decoded_labels = tokenizer.decode(
    label_ids,
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\nlabels 디코딩 결과 (-100 제외):")
print(decoded_labels)


labels 디코딩 결과 (-100 제외):
{'법령용어정의': 'allotment은 배당, 부담액, 분담, 분담액, 분배, 할당을 의미합니다. 즉, 어떤 자원이나 비용을 여러 부분으로 나누어 할당하는 것을 말합니다. 예를 들어, 회사의 이익을 주주들에게 배당하는 경우나, 프로젝트 비용을 여러 부서에 분담하는 상황에서 사용될 수 있습니다.', '출처': None, '출처유무': False}<|eot_id|>


## 5. 어텐션 마스크 확인

In [18]:
# 데이터의 최대 길이 제한
max_seq_length = 8192
# 0번과 1번 데이터의 길이 확인
example0 = train_dataset[0]
example1 = train_dataset[1]
# 개별 길이 확인 (토큰화 후)
tokenized0 = tokenizer(
    # 전체 처리 과정과 동일하게 전체 대화를 토큰화
    "<|begin_of_text|>" + "".join([f"<|start_header_id|>{msg['role']}<|end_header_id|>\n{msg['content'].strip()}<|eot_id|>" for msg in example0["messages"]]),
    truncation=True,
    max_length=max_seq_length,
    padding=False,
    return_tensors=None,
)
tokenized1 = tokenizer(
    # 전체 처리 과정과 동일하게 전체 대화를 토큰화
    "<|begin_of_text|>" + "".join([f"<|start_header_id|>{msg['role']}<|end_header_id|>\n{msg['content'].strip()}<|eot_id|>" for msg in example1["messages"]]),
    truncation=True,
    max_length=max_seq_length,
    padding=False,
    return_tensors=None,
)
print(f"0번 데이터 길이: {len(tokenized0['input_ids'])}")
print(f"1번 데이터 길이: {len(tokenized1['input_ids'])}")
# 배치로 처리하여 어텐션 마스크 비교
batch = collate_fn([example0, example1])
print("\n배치 처리 후:")
print(f"입력 ID 형태: {batch['input_ids'].shape}")
print(f"어텐션 마스크 형태: {batch['attention_mask'].shape}")
# 각 샘플의 어텐션 마스크 합계 (실제 토큰 수 확인)
print(f"0번 샘플 어텐션 마스크 합계: {batch['attention_mask'][0].sum().item()}")
print(f"1번 샘플 어텐션 마스크 합계: {batch['attention_mask'][1].sum().item()}")
# 0번 샘플과 1번 샘플의 어텐션 마스크가 다른지 확인
masks_different = not torch.equal(batch['attention_mask'][0], batch['attention_mask'][1])
print(f"\n0번과 1번 샘플의 어텐션 마스크가 다른가요? {masks_different}")
# 어텐션 마스크 패턴 시각화 (처음 20개와 마지막 20개 토큰)
print("\n0번 샘플 어텐션 마스크:", batch['attention_mask'][0].tolist())
print("1번 샘플 어텐션 마스크:", batch['attention_mask'][1].tolist())
# 배치 내에서 가장 긴 시퀀스 길이 구하기
max_length_in_batch = max(len(tokenized0['input_ids']), len(tokenized1['input_ids']))
print(f"\n배치 내 최대 길이: {max_length_in_batch}")
print(f"0번 샘플 어텐션 마스크 합계 (실제 토큰 수): {batch['attention_mask'][0].sum().item()}")
print(f"1번 샘플 어텐션 마스크 합계 (실제 토큰 수): {batch['attention_mask'][1].sum().item()}")
print(f"0번 샘플 어텐션 마스크 1의 개수: {batch['attention_mask'][0].sum().item()}")
print(f"0번 샘플 어텐션 마스크 0의 개수: {(batch['attention_mask'][0] == 0).sum().item()}")
print(f"1번 샘플 어텐션 마스크 1의 개수: {batch['attention_mask'][1].sum().item()}")
print(f"1번 샘플 어텐션 마스크 0의 개수: {(batch['attention_mask'][1] == 0).sum().item()}")
# 결과 검증: 긴 샘플은 모든 어텐션 마스크가 1이고, 짧은 샘플은 일부만 1이어야 함

0번 데이터 길이: 1107
1번 데이터 길이: 1091

배치 처리 후:
입력 ID 형태: torch.Size([2, 1107])
어텐션 마스크 형태: torch.Size([2, 1107])
0번 샘플 어텐션 마스크 합계: 1107
1번 샘플 어텐션 마스크 합계: 1091

0번과 1번 샘플의 어텐션 마스크가 다른가요? True

0번 샘플 어텐션 마스크: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

## 6. 전처리 이해하기

## 7. 학습하기

In [19]:
trainer = SFTTrainer(
    model=model,
    args=args,
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이 설정
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [19]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()   # 최종 모델을 저장

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,1.364000
20,0.887000
30,0.909800
40,0.826000
50,0.838800
60,0.805800
70,0.902500
80,0.832800
90,0.854000
100,0.732500


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.11/dist-pac

## 8. 테스트 데이터 준비하기

실제 모델에 입력을 넣을 때에는 입력의 뒤에 `<|start_header_id|>assistant<|end_header_id|>\n`가 부착되어서 넣는 것이 좋습니다. 그러면 모델이 조금 더 안정적으로 답변을 생성합니다.

In [20]:
prompt_lst = []
label_lst = []

for messages in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    input = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[0] + '<|start_header_id|>assistant<|end_header_id|>\n'
    label = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[1].split('<|eot_id|>')[0]
    prompt_lst.append(input)
    label_lst.append(label)

In [21]:
print(prompt_lst[100])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 법령 상담을 위한 챗봇 데이터를 생성하는 데이터 전문가입니다.

당신의 역할은 아래 두 가지 작업을 수행하는 것입니다:

1. 주어진 "법령 용어(법령정의사전)"를 기반으로, 고객이 자연스럽게 질문할 법한 문장을 생성하세요.
2. 주어진 "법령 용어"의 "사전식 풀이"를 바탕으로, 고객에게 친절하고 정확하게 설명하는 답변을 작성하세요.

---

[1] 질문 문장 생성 규칙:
- 말투는 자연스럽고 일상적인 질문 형태로 작성할 것
- 문장은 짧고 간결하게, 대략적인 수준으로 작성할 것
- 다음과 같은 표현을 활용할 것: "뭐예요?", "뭔가요?", "설명해 주세요."
- 용어만 나열하지 말고, 반드시 문장 형태로 만들 것
- 사람을 지칭하는 경우에는 "누구인가요?", "누구를 말하는 건가요?"처럼 질문할 것
- 반드시 질문 문장만 출력하고, 다른 설명이나 안내 문구는 출력하지 말 것

예시:
- 입력: 1인당 GNI → 출력: 1인당 GNI가 뭔가요?
- 입력: 1종 금융투자업자 → 출력: 1종 금융투자업자는 누구를 말하는 건가요?

---

[2] 정의 설명 및 출처 처리 규칙:
- 두 번째 작업의 결과는 반드시 파이썬의 dictionary 형식으로 출력해야 합니다.
- 큰따옴표(") 안에 작은따옴표(')나 다른 따옴표를 넣지 마세요. JSON 파싱 오류가 발생합니다.
- 해당 항목이 없을 경우에는 "없음"이라는 표현 대신, 빈 문자열("") 또는 빈 리스트([])를 사용해야 합니다.

요구 조건:
- 딱딱하거나 기계적인 문장은 피하고, 자연스럽고 격식 있는 설명을 작성하세요.
- 어려운 정의는 2~3문장으로 풀어 설명해 주세요. 단, 중복된 표현은 삼가세요.
- 정의 설명 결과는 "법령용어정의"라는 키에 담아야 합니다.

출처 관련 규칙:
- 출처가 없을 경우:
  {"법령용어정의": "...", "출처유무": false}

- 출처가 있을 경

In [22]:
print(label_lst[100])


{'법령용어정의': "abidance는 '준수'라는 의미를 가지고 있습니다. 이는 법이나 규칙을 따르는 것을 의미합니다. 예를 들어, 교통법규를 준수한다는 것은 교통 신호를 지키고 제한 속도를 준수하는 것을 포함합니다.", '출처': None, '출처유무': False}


## 9. 파인튜닝 모델 테스트

In [23]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import  AutoTokenizer, pipeline

In [24]:
peft_model_id = "llama3-8b-qa-ko/checkpoint-600"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

In [25]:
eos_token = tokenizer("<|eot_id|>",add_special_tokens=False)["input_ids"][0]

In [26]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [27]:
for prompt, label in zip(prompt_lst[10:15], label_lst[10:15]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


    response:
{'법령용어정의': '119감염관리실은 감염병의 발생, 확산, 예방, 대응을 위한 감염관리 및 정보관리 업무를 수행하는 기관을 의미합니다. 쉽게 말해, 감염병에 대한 정보를 수집하고 관리하여 대응 전략을 세우는 역할을 하는 곳입니다.', '출처': ['질병관리본부 감염병관리규정'], '출처유무': True}
    label:

{'법령용어정의': '119감염관리실은 감염예방 및 관리를 위해 구조ㆍ구급대원들이 장비 재처리, 물품 및 의약품 보관, 의료폐기물 관리 등을 할 수 있도록 설치된 공간을 의미합니다. 쉽게 말해, 구조와 구급 활동 중 발생할 수 있는 감염 위험을 줄이기 위해 마련된 전문적인 관리 공간이라고 생각하시면 됩니다.', '출처': ['119감염관리실 설치·운영에 관한 규정'], '출처유무': True}
--------------------------------------------------
    response:
{'법령용어정의': '119구급대는 응급구급을 제공하는 구급대와 구급차를 의미합니다. 즉, 응급 상황에서 환자를 신속하게 치료하고 병원으로 이송하기 위해 출동하는 팀을 말합니다. 이들은 응급 상황에 대비하여 필요한 의료 장비와 약품을 갖추고 있어야 하며, 환자의 생명을 구하기 위해 최선을 다합니다.', '출처': ['구급대 및 구급차 운영에 관한 규정'], '출처유무': True}
    label:

{'법령용어정의': '119구급대는 구급활동에 필요한 장비를 갖추고 소방공무원으로 구성된 단위조직을 의미합니다. 쉽게 말해, 응급 상황에서 신속하게 대응할 수 있도록 준비된 팀이라고 생각하시면 됩니다.', '출처': ['119구조ㆍ구급에 관한 법률'], '출처유무': True}
--------------------------------------------------
    response:
{'법령용어정의': '119구조견은 구조물의 안전성을 평가하고, 필요한 경우 구조물의 안전을 보장하기 위해 사용되는 전

## 10. 기본 모델 테스트

이번에는 LoRA Adapter를 merge하지 않은 기본 모델로 테스트 데이터에 대해서 인퍼런스해보겠습니다.


In [28]:
base_model_id = "NCSOFT/Llama-VARCO-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [29]:
for prompt, label in zip(prompt_lst[10:15], label_lst[10:15]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

    response:
119감염관리실은 공공보건에 관련된 질병 및 감염병 발생 시 신속하고 효과적인 대응을 위해 설립된 기관입니다.

법령용어정의:
{
  "119감염관리실": "공공보건에 관련된 질병 및 감염병 발생 시 신속하고 효과적인 대응을 위해 설립된 기관입니다.",
  "출처유무": true,
  "출처": ["질병관리청"]
}

설명:
119감염관리실은 질병관리청(예전에는 보건복지부)에 속해 있는 기관으로, 주로 전염병이나 감염병 등 공중보건 위협 요소에 대응하기 위해 설립되었습니다. 

이 기관은 질병 발생 시 즉각적인 조치를 취하고, 관련 정보를 수집, 분석, 전달함으로써 질병 확산을 최소화하고 대중보건을 보호하는 역할을 합니다. 

119감염관리실의 주요 업무로는 감염병 발생 사례 신고 접수, 환자 추적 및 격리 조치, 의료진 및 시민 교육 등이 포함됩니다. 

이 기관의 이름에서 알 수 있듯이, '119'는 응급 상황에 대한 신고 전화번호를 상징하며, '감염관리실'은 그들이 주로 다루는 감염병 관리라는 의미를 내포하고 있습니다. 

119감염관리실은 각 지역마다 존재하며, 현지 질병 발생 시 현장에서 직접 대응하고 중앙 정부와 협력하여 전반적인 대응 방안을 수립합니다.
    label:

{'법령용어정의': '119감염관리실은 감염예방 및 관리를 위해 구조ㆍ구급대원들이 장비 재처리, 물품 및 의약품 보관, 의료폐기물 관리 등을 할 수 있도록 설치된 공간을 의미합니다. 쉽게 말해, 구조와 구급 활동 중 발생할 수 있는 감염 위험을 줄이기 위해 마련된 전문적인 관리 공간이라고 생각하시면 됩니다.', '출처': ['119감염관리실 설치·운영에 관한 규정'], '출처유무': True}
--------------------------------------------------
    response:
119구급대는 응급상황 발생 시 즉각 현장으로 출동하여 환자 치료를 제공하는 전문 의료팀을 말합니다.

법령용어정의:
{
  "119구급대": "응